<a href="https://colab.research.google.com/github/edwarhman/100-days-of-code/blob/master/practica_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Es necesario importar la librería numpy

In [ ]:
import numpy as np

# Métodos directos

## Cálculo de Triangular superior

dada una matriz A de dimensiones n x n y un vector b de tamaño n, retorna la matriz superior de A y las transformaciones pertinentes al vector b.

### Entrada:

    A: Matriz de dimensiones n x n
    b: Vector de tamaño n

### Salida:

    B: Matriz superior de dimensiones n x n
    c: vector transformación de tamaño n

In [ ]:
def triangsupMat(A, b):
  '''
  Ecuentra la matriz triangular superior de A y la transformación de b correspondiente

  Args:
    A (array): matriz
    b (array): vector

  Returns:
    B (array): matriz triangular superior
    c (array): vector
  '''
  (n,m) = A.shape
  o = b.size
  B = []

  if (m!=n):
    print('La matriz no es cuadrada')
    return
  if(n!=o):
    print('El sistema no es compatible determinado')
    return

  B = np.copy(A)
  c = np.copy(b)

  # pivoteo parcial
  for j in range(0,n-1):
    k = np.argmax(np.abs(B[j:n,j])) # maximo pivote por columna
    k=k+j
    # intercambio de filas
    faux = np.copy(B[j,:])
    B[j,:] = np.copy(B[k,:])
    B[k,:] = np.copy(faux)
    # intercambio elementos del vector
    caux = c[j]
    c[j] = c[k]
    c[k] = caux
    for i in range(j+1,n):
      mu = B[i,j]/B[j,j] # calculo de multiplicadores
      B[i,:] = B[i,:] - (mu*B[j,:])
      c[i] = c[i] - (mu*c[j])
  return B, c

## Resolver por sustitución

Resuelve un sistema de ecuaciones lineales mediante la sustitucion de variables.

### Argumentos

    A (np.array): Matriz de ecuaciones.
    b (np.array): Vector de constantes.
    forward (bool): Direccion de la resolucion. True: hacia arriba, False: hacia abajo.

### Resultado
    x: Vector de soluciones del sistema.

In [ ]:
def resolverPorSustitucion(A, b, forward=False):
  """Resuelve un sistema de ecuaciones lineales mediante la sustitucion
  de variables.

  Args:
    A (np.array): Matriz de ecuaciones.
    b (np.array): Vector de constantes.
    forward (bool): Direccion de la resolucion. True: hacia arriba, False: hacia abajo.

  Returns:
    np.array: Vector de solucion.
  """

  n = b.size
  x = np.zeros(n)
  rango = range(0, n) if forward else range(n-1, -1, -1)

  for i in rango:
    x[i] = (b[i] - (np.sum(A[i,:] * x))) / A[i, i]

  return x

## Eliminación Gaussiana

Resuelve el sistema de ecuaciones $AX = B$ mediante el método de Gauss-Jordan

### Argumentos
  A es una matriz de dimensión n x n
  
  b es vector de tamaño n  

### Resultado
Devuelve un vector de solución de $Ax = B$

In [ ]:
def resolverSistemaEliminacionGaussiana(A, b):
    '''
    Resuelve el sistema de ecuaciones AX = B mediante el método de Gauss-Jordan

    Args:
    A (array): matriz de ecuaciones
    b (array): vector de constantes

    Returns:
    array: vector de solución
    '''
    B, c = triangsupMat(A,b)

    return resolverPorSustitucion(B, c)

## Factorización LU

Descompone una matriz cuadrada en su factorización LU.

### Argunmentos

* A: matriz de la matriz de ecuaciones

### Resultado

* L: matriz triangular inferior L
* U: matriz triangular superior U


In [ ]:
def factolizarLU(A):
    '''
    Factorizacion LU de una matriz

    Args:
        A (array): matriz a factorizar
    Returns:
        L (array): matriz L de la factorizacion LU
        U (array): matriz U de la factorizacion LU
    '''
    (n,m) = A.shape


    if (m!=n):
        print ('La matriz no es cuadrada')
        return

    # inicializacion de matrizes
    U = []
    L = np.zeros((n,n))

    U = np.copy(A)
    for j in range(0,n-1):
        k = np.argmax(np.abs(U[j:n,j])) # maximo pivote por columna
        k=k+j

        for i in range(j+1,n):
             mu = U[i,j]/U[j,j]    # calculo de multiplicadores
             U[i,:] = U[i,:] - (mu*U[j,:]) # multiplicacion por el multiplicador
             L[i,j] = mu # asignacion de multiplicador

    L = np.identity(n) + L # sumar matriz identidad
    return L, U

## Resolver Sistema mediante Factorización LU

Resuelve el sistema de ecuaciones lineales mediante el método de factorización LU.

### Argumentos

* `A`: matriz del sistema de ecuaciones lineales
* `b`: vector de constantes del sistema de ecuaciones lineales

### Resultado

* `x`: vector de solución del sistema de ecuaciones lineales

In [ ]:
def resolverSistemaLU(A, b):
  '''
  Resuelve un sistema de ecuaciones lineales usando el metodo LU

  Args:
    A (array): matriz de la ecuacion
    b (array): vector de la ecuacion

  Returns:
    array: vector de la solucion
  '''
  L, U = factolizarLU(A)
  y = resolverPorSustitucion(L,b, forward=True)
  return resolverPorSustitucion(U, y)

## Factorización PALU

Descompone una matriz cuadrada en su factorización LU usando permutaciones de filas.

### Argumentos

* `A`: matriz cuadrada

### Resultados

* `L`: matriz L
* `U`: matriz U
* `P`: permutaciones de filas

In [ ]:
def factolizarPALU(A):
    '''
    Factorizacion de matrizes cuadradas por LU y permutaciones de filas

    Args:
        A (array): matriz a factorizar
    Returns:
        P (array): matriz de permutaciones
        L (array): matriz L
        U (array): matriz U
    '''
    (n,m) = A.shape


    if (m!=n):
        print ('La matriz no es cuadrada')
        return

    U = []
    P = np.identity(n)
    L = np.zeros((n,n))

    U = np.copy(A)
    for j in range(0,n-1):
        k = np.argmax(np.abs(U[j:n,j])) # maximo pivote por columna
        k=k+j

        # Permutar matriz Superior
        faux = np.copy(U[j,:])
        U[j,:] = np.copy(U[k,:])
        U[k,:] = np.copy(faux)

        # Registrar permutaciones
        paux = np.copy(P[j,:])
        P[j,:] = np.copy(P[k,:])
        P[k,:] = np.copy(paux)

        # Permutar matriz inferior
        laux = np.copy(L[j,:])
        L[j,:] = np.copy(L[k,:])
        L[k,:] = np.copy(laux)


        for i in range(j+1,n):
             mu = U[i,j]/U[j,j]    # calculo de multiplicadores
             U[i,:] = U[i,:] - (mu*U[j,:]) # multiplicacion por multiplicadores
             L[i,j] = mu # registro de multiplicadores

    L = np.identity(n) + L # sumar matriz identidad
    return P, L, U

## Resolver Sistema mediante factorización PALU

Resuelve el sistema de ecuaciones lineales mediante el método de factorización PALU.

### Argumentos

* `A`: matriz de la ecuación a resolver.
* `b`: vector de la ecuación a resolver.

### Resultado

* `x`: vector de solución.

In [ ]:
def resolverSistemaPALU(A, b):
  '''
  Resuelve un sistema de ecuaciones lineales mediante el método de PALU

  Args:
    A (array): matriz de ecuaciones
    b (array): vector de constantes

  Returns:
    array: vector de solución
  '''
  P, L, U = factolizarPALU(A)
  y = resolverPorSustitucion(L,np.matmul(P,b), forward=True)
  return resolverPorSustitucion(U, y)
  # return np.round(resolverPorSustitucion(U, y))


## Ejemplos

In [ ]:

# @title Ejemplo lamina SistemasLineales pag. 29
A = np.array([
    [1.,0.,1.],
    [2.,3.,-1.],
    [1.,2.,1.]
])

b = np.array([2.,7.,6.])

x = resolverSistemaLU(A, b)
print('LU')
print(x)
comprobacion = np.matmul(A, x)
print(comprobacion)

x = resolverSistemaPALU(A, b)
print('PALU')
print(x)
comprobacion = np.matmul(A, x)
print(comprobacion)

print('Eliminación Gaussiana')
x = resolverSistemaEliminacionGaussiana(A, b)
print(x)
comprobacion = np.matmul(A, x)
print(comprobacion)


LU
[1. 2. 1.]
[2. 7. 6.]
PALU
[1. 2. 1.]
[2. 7. 6.]
Eliminación Gaussiana
[1. 2. 1.]
[2. 7. 6.]


In [ ]:

#@title ejemplo libro burden faires pag. 270

A = np.array([
    [1.0, 1.0,0.0,3.0],
    [2.0,1.0,-1.0,1.0],
    [3.0,-1.0,-1.0,2.0],
    [-1.0,2.0,3.0,-1.0]
])

b = np.array([4,1,-3,4])

print(A, b)

resultado = resolverSistemaLU(A, b)
print('\n')
print('LU')
print(resultado)
comprobacion = np.matmul(A, resultado)
print(comprobacion)
resultado = resolverSistemaPALU(A, b)
print('\n')
print('PALU')
print(np.round(resultado, 2))
comprobacion = np.matmul(A, resultado)
print(comprobacion)
print('\n')
print('Eliminación Gaussiana')
x = resolverSistemaEliminacionGaussiana(A, b)
print(np.round(x, 2))
comprobacion = np.matmul(A, x)
print(comprobacion)


[[ 1.  1.  0.  3.]
 [ 2.  1. -1.  1.]
 [ 3. -1. -1.  2.]
 [-1.  2.  3. -1.]] [ 4  1 -3  4]


LU
[-1.  2.  0.  1.]
[ 4.  1. -3.  4.]


PALU
[-1.  2. -0.  1.]
[ 4.  1. -3.  4.]


Eliminación Gaussiana
[-0.86  1.95  0.    0.77]
[ 3.4  1.  -3.   4. ]


In [ ]:
#@title ejemplo libro burden faires pag. 275

A = np.array([
    [1.0, -1.0,2.0,-1.0],
    [2.0,-2.0,3.0,-3.0],
    [1.0,1.0,1.0,0.0],
    [1.0,-1.0,4.0,3.0]
])

b = np.array([-8,-20,-2,4])

print(A, b)

resultado = resolverSistemaLU(A, b)
print('\n')
print('LU')
print(resultado)
comprobacion = np.matmul(A, resultado)
print(comprobacion)
resultado = resolverSistemaPALU(A, b)
print('\n')
print('PALU')
print(resultado)
comprobacion = np.matmul(A, resultado)
print(comprobacion)
print('\n')
print('Eliminación Gaussiana')
x = resolverSistemaEliminacionGaussiana(A, b)
print(x)
comprobacion = np.matmul(A, x)
print(comprobacion)

[[ 1. -1.  2. -1.]
 [ 2. -2.  3. -3.]
 [ 1.  1.  1.  0.]
 [ 1. -1.  4.  3.]] [ -8 -20  -2   4]


LU
[nan nan nan nan]
[nan nan nan nan]


PALU
[-7.  3.  2.  2.]
[ -8. -20.  -2.   4.]


Eliminación Gaussiana
[-13.    5.4   5.6  -0. ]
[ -7.2 -20.   -2.    4. ]


<ipython-input-26-223a351b75af>:28: RuntimeWarning: divide by zero encountered in scalar divide
  mu = U[i,j]/U[j,j]    # calculo de multiplicadores
<ipython-input-26-223a351b75af>:29: RuntimeWarning: invalid value encountered in multiply
  U[i,:] = U[i,:] - (mu*U[j,:]) # multiplicacion por el multiplicador
<ipython-input-26-223a351b75af>:28: RuntimeWarning: invalid value encountered in scalar divide
  mu = U[i,j]/U[j,j]    # calculo de multiplicadores
<ipython-input-24-e61f77049430>:19: RuntimeWarning: invalid value encountered in multiply
  x[i] = (b[i] - (np.sum(A[i,:] * x))) / A[i, i]


In [ ]:
# prueba LU Libro ingles 409

A = np.array([
    [1., 1., 0., 3.],
    [2., 1., -1., 1.],
    [3., -1., -1., 2.],
    [-1., 2., 3., -1.]
])
b = np.array([4., -7., 13., -13.])
print(A, b)

x = resolverSistemaLU(A, b)

print('\n')
print('metodo LU')
print(x)
comprobacion = np.matmul(A, x)
print(comprobacion)
resultado = resolverSistemaPALU(A, b)
print('\n')
print('PALU')
print(resultado)
comprobacion = np.matmul(A, resultado)
print(comprobacion)
print('\n')
print('Eliminación Gaussiana')
x = resolverSistemaEliminacionGaussiana(A, b)
print(x)
comprobacion = np.matmul(A, x)
print(comprobacion)
print('\n')

[[ 1.  1.  0.  3.]
 [ 2.  1. -1.  1.]
 [ 3. -1. -1.  2.]
 [-1.  2.  3. -1.]] [  4.  -7.  13. -13.]


metodo LU
[-0.35897436 -8.1025641   2.33333333  4.15384615]
[  4.  -7.  13. -13.]


PALU
[-0.35897436 -8.1025641   2.33333333  4.15384615]
[  4.  -7.  13. -13.]


Eliminación Gaussiana
[-0.35897436 -8.1025641   2.33333333  4.15384615]
[  4.  -7.  13. -13.]




# Métodos iterativos

## Comprobar Matriz Diagonal Dominante

Comprueba si una matriz es diagonal dominante o estrictamente diagonal dominante.

### Argumentos

    A: Matriz cuadrada n x n a evaluar
    estricto: false para evaluar si es diagonal dominante o true para evaluar si es estrictamente diagonal dominante

### Resultado

    valor booleano indicando si es diagonal dominante o no

In [ ]:
def comprobarMatrizDiagonalDominante(A, estricto = False):
    '''
    Devuelve True si la matriz es diagonal dominante, False en caso contrario

    '''
    n = A.shape[1]

    for i in range(0,n): # itera a través de cada fila
         # obtiene la suma absoluta de todos los elementos de la fila menos el elemento diagonal
        suma = np.sum(np.abs(A[i,:])) - np.abs(A[i,i])
        # si la suma es mayor a la diagonal absoluta, la matriz no es diagonal dominante
        if(suma >= np.abs(A[i,i]) if estricto else suma > np.abs(A[i,i])):
            return False
    return True

## Comprobar Convergencia

Mediante el uso del radio espectral de una matriz, determina si un sistema de
ecuaciones lineales converge a una solución o no.

### Argumentos

    A: Matriz cuadrada n x n a evaluar

### Resultado

    valor booleano indicando si el sistema converge a una solución o no

In [ ]:
def comprobarConvergenciaSolucion(A):
    valores, vectores = np.linalg.eig(A) # determina los valores propios de la matriz
    radioEspectral = np.max(np.abs(valores)) # determina el radio espectral de la matriz
    print('Radio espectral: ', radioEspectral, 'converge:', radioEspectral <= 1)
    return radioEspectral < 1

## Método Jacobi para resolver sistemas de ecuaciones lineales

Resuelve un sistema de ecuaciones lineales de la forma:

$$
\begin{align}
a_{11}x_1 + a_{12}x_2 + \cdots + a_{1n}x_n &= b_1 \\
a_{21}x_1 + a_{22}x_2 + \cdots + a_{2n}x_n &= b_2 \\
\vdots \\
a_{m1}x_1 + a_{m2}x_2 + \cdots + a_{mn}x_n &= b_m
\end{align}
$$

con $a_{ij}, b_i \in \mathbb{R}$ y $x_i \in \mathbb{R}$.

### Argumentos

* $a_{ij}$ es la matriz de coeficientes del sistema.
* $b_i$ es el vector de resultados del sistema.
* $iteraciones$ es el número de iteraciones que se realizarán.

### Resultado

* $x_i$ es el vector de soluciones del sistema.



In [ ]:
#
def resolverJacobi(A, b, iteraciones):
    '''
    Resuelve la matriz A de orden n x n mediante el método de Jacobi.

    Args:
        A (np.array): matriz de orden n x n
        b (np.array): vector de orden n
        iteraciones (int): número de iteraciones

    Returns:
        np.array: solución de la matriz A
    '''
    m, n = A.shape

    if(m!=n):
        print('La matriz no es cuadrada')
        return

    # esEstrictamenteDiagonalDominante = comprobarMatrizDiagonalDominante(A, estricto=True)
    # print(' es edd', esEstrictamenteDiagonalDominante)
    # if(esEstrictamenteDiagonalDominante == False):
    #     print('La matriz no es estrictamente diagonal dominante')
    #     return

    # construcción de parametros para aplicar el metodo de Jacobi
    D = np.diagflat([A.diagonal()])
    Au = (np.triu(A) - D) * -1
    Al = (np.tril(A) - D ) * -1
    print(np.matmul(np.linalg.inv(D), (Al + Au)))
    if(comprobarConvergenciaSolucion(np.matmul(np.linalg.inv(D), (Al + Au))) == False):
        print('La matriz no converge a una solución')
        return

    x = np.ones(n)

    # iteración del método de Jacobi
    for n in range(0, iteraciones):
        # encontrar x_k+1 mediante la ecuación de Jacobi
        x = np.matmul(np.linalg.inv(D), np.matmul(Al + Au, x)) + np.matmul(np.linalg.inv(D), b)

    return x


## Método de Gauss-Seidel para resolver sistemas de ecuaciones lineales

Resuelve un sistema de ecuaciones lineales de la forma:

$$
\begin{align}
a_{11}x_1 + a_{12}x_2 + \cdots + a_{1n}x_n &= b_1 \\
a_{21}x_1 + a_{22}x_2 + \cdots + a_{2n}x_n &= b_2 \\
\vdots \\
a_{m1}x_1 + a_{m2}x_2 + \cdots + a_{mn}x_n &= b_m
\end{align}
$$

con $a_{ij}, b_i \in \mathbb{R}$ y $x_i \in \mathbb{R}$.

### Argumentos

* **$a_{ij}$** es la matriz de coeficientes del sistema.
* **$b_i$** es el vector de resultados del sistema.
* **$iteraciones$** es el número de iteraciones que se realizarán.

### Resultado

* **$x_i$** es el vector de soluciones del sistema.


In [ ]:
def resolverGaussSeidel(A, b, iteraciones):
    '''
    Resuelve un sistema de ecuaciones lineales mediante el método de Gauss-Seidel.

    Args:
        A (np.array): Matriz de la ecuación a resolver.
        b (np.array): Vector de los constantes de la ecuación.
        iteraciones (int): Número de iteraciones del método.

    Returns:
        np.array: Vector de solución de la ecuación.
    '''

    m, n = A.shape

    if(m!=n):
        print('La matriz no es cuadrada')
        return

    # if(comprobarMatrizDiagonalDominante(A, estricto=True) == False):
    #     print('La matriz no es estrictamente diagonal dominante')
    #     return

    D = np.diagflat([A.diagonal()])
    Au = (np.triu(A) - D) * -1
    Al = (np.tril(A) - D ) * -1

    if(comprobarConvergenciaSolucion(np.matmul(np.linalg.inv(D - Al), Au)) == False):
        print('La matriz no converge a una solución')
        return

    x = np.ones(n)


    for n in range(0, iteraciones):
        # encontrar solución x_k+1 usando la ecuación de Gauss-Seidel
        x = np.matmul(np.linalg.inv(D - Al), np.matmul(Au, x)) + np.matmul(np.linalg.inv(D - Al), b)

    return x

## Método SOR para resolver sistemas de ecuaciones lineales

Resuelve un sistema de ecuaciones lineales de la forma:

$$
\begin{align}
a_{11}x_1 + a_{12}x_2 + \cdots + a_{1n}x_n &= b_1 \\
a_{21}x_1 + a_{22}x_2 + \cdots + a_{2n}x_n &= b_2 \\
\vdots \\
a_{m1}x_1 + a_{m2}x_2 + \cdots + a_{mn}x_n &= b_m
\end{align}
$$

con $a_{ij}, b_i \in \mathbb{R}$ y $x_i \in \mathbb{R}$.

### Argumentos

- $a_{ij}$ son los coeficientes de la ecuación $i$ en la ecuación $j$.
- $b_i$ son los constantes de la ecuación $i$.
- $iteraciones$ es el número de iteraciones que se realizarán.
- $w$ es el parámetro de relajación para acelerar la convergencia.

### Resultado

- $x_{k+1}$ es el resultado del sistema de ecuaciones lineales.

In [ ]:
def resolverSOR(A, b, iteraciones, w):
    '''
    Resuelve el sistema de ecuaciones AX = B usando el método de SOR.

    Args:
        A (array): matriz de la ecuación AX = B
        b (array): vector de la ecuación AX = B
        iteraciones (int): número de iteraciones
        w (float): peso de la matriz diagonal dominante

    Returns:
        array: vector de la solución de la ecuación AX = B
    '''
    m, n = A.shape

    if(m!=n):
        print('La matriz no es cuadrada')
        return

    # if(comprobarMatrizDiagonalDominante(A, estricto=True) == False):
    #     print('La matriz no es estrictamente diagonal dominante')
    #     return

    # construir los parametros para aplicar el algoritmo de SOR
    D = np.diagflat([A.diagonal()])
    Au = (np.triu(A) - D) * -1
    Al = (np.tril(A) - D ) * -1

    if(comprobarConvergenciaSolucion(np.matmul(np.linalg.inv(D - w*Al), w*Au + (1-w)*D)) == False):
        print('La matriz no converge a una solución')
        return

    x = np.ones(n)

    for n in range(0, iteraciones):
        # calcular la solución x_k+1 de la ecuación AX = B usando el algoritmo de SOR
        x = np.matmul(np.linalg.inv(D - w*Al), np.matmul(w*Au + (1-w)*D, x)) + np.matmul(np.linalg.inv(D - w*Al), w*b)

    return x

## Ejemplos

In [ ]:
A = np.array([
    [5, -1, 3],
    [3, 6, 2],
    [2, 2, 4]
])

b = np.array([-4,11,6])
x = np.array([-1, 2, 1])

print(A)
print(b)
print(x)

print('Jacobi')
resultado = resolverJacobi(A, b, 60)
print(resultado)
print('Gauss-Seidel')
resultado = resolverGaussSeidel(A, b, 16)
print(resultado)
print('SOR')
resultado = resolverSOR(A, b, 300, 0.09999999999999)
print(resultado)

[[ 5 -1  3]
 [ 3  6  2]
 [ 2  2  4]]
[-4 11  6]
[-1  2  1]
Jacobi
Radio espectral:  0.7261724292547672 converge: True
[-1.  2.  1.]
Gauss-Seidel
Radio espectral:  0.32062898671428325 converge: True
[-1.  2.  1.]
SOR
Radio espectral:  0.9344363965398466 converge: True
[-1.  2.  1.]


In [ ]:
A = np.array([
    [4., 3., 0.],
    [3., 4., -1.],
    [0., -1., 4.]
])

b = np.array([24., 30., -24.])
x0 = np.array([1., 1., 1.])
w= 1.25
resultado = resolverSOR(A, b, 100, w)
print(resultado)

Radio espectral:  0.25 converge: True
[ 3.  4. -5.]


In [ ]:
Al = np.tril(A, -1)
Au = np.triu(A)

x1 = np.array([[1.], [1.], [1.]])

print(np.matmul(A, x1))
print(np.matmul(A, x0))
# print(np.matmul(x1, A))
print(np.matmul(x0, A))

[[7.]
 [6.]
 [3.]]
[7. 6. 3.]
[7. 6. 3.]


# Practica

## Actividad 1

### Construcción del sistema

Construimos un sistema de ecuaciones de orden m, con m variables, y m condicioness.

In [ ]:
# sistema ejercicio 1
m = 50
b = np.linspace(1, 50)

ones = np.ones(m-1)
A = np.zeros((m,m))

A = A + np.diagflat([b])
A = A - np.diagflat([ones], 1)
A = A + np.diagflat([ones], -1)
b = b*-1

print('A')
print(A)
print('b')
print(b)

A
[[ 1. -1.  0. ...  0.  0.  0.]
 [ 1.  2. -1. ...  0.  0.  0.]
 [ 0.  1.  3. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 48. -1.  0.]
 [ 0.  0.  0. ...  1. 49. -1.]
 [ 0.  0.  0. ...  0.  1. 50.]]
b
[ -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12. -13. -14.
 -15. -16. -17. -18. -19. -20. -21. -22. -23. -24. -25. -26. -27. -28.
 -29. -30. -31. -32. -33. -34. -35. -36. -37. -38. -39. -40. -41. -42.
 -43. -44. -45. -46. -47. -48. -49. -50.]


### Eliminación Gaussiana

In [ ]:
x = resolverSistemaEliminacionGaussiana(A, b)
comprobacion = np.matmul(A, x)

print('Resultado')
print(x)
print('Comprobación')
print(comprobacion)

Resultado
[-1.69777466 -0.69777466 -1.09332397 -0.97774658 -1.00431029 -0.99929804
 -1.00009854 -0.99998785 -1.00000134 -0.99999987 -1.00000001 -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -0.99999982 -0.99999151
 -0.99959218 -0.98000816]
Comprobación
[ -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12. -13. -14.
 -15. -16. -17. -18. -19. -20. -21. -22. -23. -24. -25. -26. -27. -28.
 -29. -30. -31. -32. -33. -34. -35. -36. -37. -38. -39. -40. -41. -42.
 -43. -44. -45. -46. -47. -48. -49. -50.]


### Factorización LU

In [ ]:
x = resolverSistemaLU(A, b)
comprobacion = np.matmul(A, x)

print('resultado')
print(x)
print('comprobación')
print(comprobacion)

resultado
[-1.69777466 -0.69777466 -1.09332397 -0.97774658 -1.00431029 -0.99929804
 -1.00009854 -0.99998785 -1.00000134 -0.99999987 -1.00000001 -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -0.99999982 -0.99999151
 -0.99959218 -0.98000816]
comprobación
[ -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12. -13. -14.
 -15. -16. -17. -18. -19. -20. -21. -22. -23. -24. -25. -26. -27. -28.
 -29. -30. -31. -32. -33. -34. -35. -36. -37. -38. -39. -40. -41. -42.
 -43. -44. -45. -46. -47. -48. -49. -50.]


### Factorización PALU

In [ ]:
x = resolverSistemaPALU(A, b)
comprobacion = np.matmul(A, x)

print('Resultado')
print(x)
print('Comprobación')
print(comprobacion)

Resultado
[-1.69777466 -0.69777466 -1.09332397 -0.97774658 -1.00431029 -0.99929804
 -1.00009854 -0.99998785 -1.00000134 -0.99999987 -1.00000001 -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -0.99999982 -0.99999151
 -0.99959218 -0.98000816]
Comprobación
[ -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12. -13. -14.
 -15. -16. -17. -18. -19. -20. -21. -22. -23. -24. -25. -26. -27. -28.
 -29. -30. -31. -32. -33. -34. -35. -36. -37. -38. -39. -40. -41. -42.
 -43. -44. -45. -46. -47. -48. -49. -50.]


### Método de Jacobi

In [ ]:
x = resolverJacobi(A, b, 110)
comprobacion = np.matmul(A, x)

print('Resultado')
print(x)
print('Comprobación')
print(comprobacion)

Radio espectral:  0.8316611546312483 converge: True
Resultado
[-1.69777466 -0.69777466 -1.09332397 -0.97774658 -1.00431029 -0.99929804
 -1.00009854 -0.99998785 -1.00000134 -0.99999987 -1.00000001 -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -0.99999982 -0.99999151
 -0.99959218 -0.98000816]
Comprobación
[ -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12. -13. -14.
 -15. -16. -17. -18. -19. -20. -21. -22. -23. -24. -25. -26. -27. -28.
 -29. -30. -31. -32. -33. -34. -35. -36. -37. -38. -39. -40. -41. -42.
 -43. -44. -45. -46. -47. -48. -49. -50.]


### Método de Gauss-Siedel

In [ ]:
x = resolverGaussSeidel(A, b, 55)
comprobacion = np.matmul(A, x)

print(x)
print(comprobacion)

Radio espectral:  0.6916602761225797 converge: True
[-1.69777466 -0.69777466 -1.09332397 -0.97774658 -1.00431029 -0.99929804
 -1.00009854 -0.99998785 -1.00000134 -0.99999987 -1.00000001 -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -0.99999982 -0.99999151
 -0.99959218 -0.98000816]
[ -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12. -13. -14.
 -15. -16. -17. -18. -19. -20. -21. -22. -23. -24. -25. -26. -27. -28.
 -29. -30. -31. -32. -33. -34. -35. -36. -37. -38. -39. -40. -41. -42.
 -43. -44. -45. -46. -47. -48. -49. -50.]


### Método SOR

In [ ]:
x = resolverSOR(A, b, 45, 0.5)
comprobacion = np.matmul(A, x)

print(x)
print(comprobacion)

Radio espectral:  0.5000000000000022 converge: True
[-1.69777466 -0.69777466 -1.09332397 -0.97774658 -1.00431029 -0.99929804
 -1.00009854 -0.99998785 -1.00000134 -0.99999987 -1.00000001 -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -0.99999982 -0.99999151
 -0.99959218 -0.98000816]
[ -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12. -13. -14.
 -15. -16. -17. -18. -19. -20. -21. -22. -23. -24. -25. -26. -27. -28.
 -29. -30. -31. -32. -33. -34. -35. -36. -37. -38. -39. -40. -41. -42.
 -43. -44. -45. -46. -47. -48. -49. -50.]


## Actividad 2

### construcción del sistema

In [ ]:
# Sistema ejercicio 2

A = np.zeros((m,m))
b = np.linspace(1, 50)
A = A + np.diagflat([b])
A[:, 0] = b
A[0, 1] = -1
A[0,m-1] = -1
print('A')
print(A)
print('b')
print(b)

A
[[ 1. -1.  0. ...  0.  0. -1.]
 [ 2.  2.  0. ...  0.  0.  0.]
 [ 3.  0.  3. ...  0.  0.  0.]
 ...
 [48.  0.  0. ... 48.  0.  0.]
 [49.  0.  0. ...  0. 49.  0.]
 [50.  0.  0. ...  0.  0. 50.]]
b
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]


### Eliminación Gaussiana

In [ ]:
x = resolverSistemaEliminacionGaussiana(A, b)
comprobacion = np.matmul(A, x)

print(x)
print(comprobacion)

[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.26882631e-16  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -1.26882631e-16  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.26882631e-16
  1.22507368e-16  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46.

### Factorización LU

In [ ]:
x = resolverSistemaLU(A, b)
comprobacion = np.matmul(A, x)

print(x)
print(comprobacion)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]


### Factorización PALU

In [ ]:
x = resolverSistemaPALU(A, b)
comprobacion = np.matmul(A, x)

print(x)
print(comprobacion)

[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.26882631e-16  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -1.26882631e-16  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.26882631e-16
  1.22507368e-16  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46.

### Método de Jacobi

In [ ]:
x = resolverJacobi(A, b, 100)
comprobacion = np.matmul(A, x)

print(x)
print(comprobacion)

Radio espectral:  1.4142135623730965 converge: False
La matriz no converge a una solución


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

### Método de Gauss-Siedel

In [ ]:
x = resolverGaussSeidel(A, b, 50)
comprobacion = np.matmul(A, x)

print(x)
print(comprobacion)

Radio espectral:  2.0 converge: False
La matriz no converge a una solución


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

### Método SOR

Eligiendo W = 1:

In [ ]:
x = resolverSOR(A, b, 55, 1)
comprobacion = np.matmul(A, x)

print(x)
print(comprobacion)


Radio espectral:  2.0 converge: False
La matriz no converge a una solución


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

Eligiendo W = 0.828

In [ ]:
x = resolverSOR(A, b, 1550, 0.828)
comprobacion = np.matmul(A, x)

print('Solución')
print(np.round(x, 2))
print('Comprobación')
print(np.round(comprobacion, 2))

Radio espectral:  0.9975101566148298 converge: True
Solución
[0.96 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03
 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03
 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03
 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03]
Comprobación
[ 0.91  1.98  2.97  3.96  4.95  5.94  6.93  7.92  8.91  9.89 10.88 11.87
 12.86 13.85 14.84 15.83 16.82 17.81 18.8  19.79 20.78 21.77 22.76 23.75
 24.74 25.73 26.72 27.7  28.69 29.68 30.67 31.66 32.65 33.64 34.63 35.62
 36.61 37.6  38.59 39.58 40.57 41.56 42.55 43.54 44.53 45.51 46.5  47.49
 48.48 49.47]


In [ ]:
x = resolverSOR(A, b, 55, 0.428)
comprobacion = np.matmul(A, x)

print(x)
print(comprobacion)

Radio espectral:  0.5720000000000004 converge: True
[ 1.00000000e+00 -7.16093851e-15 -7.16093851e-15 -7.16093851e-15
 -7.32747196e-15 -7.16093851e-15 -7.29971639e-15 -7.16093851e-15
 -7.21644966e-15 -7.32747196e-15 -7.32747196e-15 -7.16093851e-15
 -7.10542736e-15 -7.29971639e-15 -7.29971639e-15 -7.16093851e-15
 -7.16093851e-15 -7.29971639e-15 -7.21644966e-15 -7.32747196e-15
 -7.10542736e-15 -7.32747196e-15 -7.21644966e-15 -7.16093851e-15
 -7.29971639e-15 -7.02216063e-15 -7.21644966e-15 -7.29971639e-15
 -7.21644966e-15 -7.29971639e-15 -7.21644966e-15 -7.16093851e-15
 -7.41073869e-15 -7.16093851e-15 -7.10542736e-15 -7.29971639e-15
 -7.41073869e-15 -7.21644966e-15 -7.29971639e-15 -7.32747196e-15
 -7.35522754e-15 -7.16093851e-15 -7.16093851e-15 -7.32747196e-15
 -7.35522754e-15 -7.21644966e-15 -7.10542736e-15 -7.16093851e-15
 -7.29971639e-15 -7.16093851e-15]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 

## Problema Evaluación

In [ ]:
A = np.array([
    [5.3, 2.3, 4.6, 2.7, 1.6, 2.2],
    [2.4, 7.8, 5.7, 8.4, 3.4, 4.2],
    [3.4, 5.6, 2.4, 1.7, 7.4, 3.9],
    [8.3, 7.5, 9.2, 6.1, 5.2, 7.9],
    [4.3,5.9,7.2,2.6,4.9,8.0],
    [9.0,2.7,4.9,4.8,6.7,4.9]

])

b = np.array([15.590, 44.020, 18.050, 43.960, 28.240, 34.490])

print(A)
print(b)

[[5.3 2.3 4.6 2.7 1.6 2.2]
 [2.4 7.8 5.7 8.4 3.4 4.2]
 [3.4 5.6 2.4 1.7 7.4 3.9]
 [8.3 7.5 9.2 6.1 5.2 7.9]
 [4.3 5.9 7.2 2.6 4.9 8. ]
 [9.  2.7 4.9 4.8 6.7 4.9]]
[15.59 44.02 18.05 43.96 28.24 34.49]


In [ ]:
x = resolverSistemaEliminacionGaussiana(A, b)
comprobacion = np.matmul(A, x)

print('resultado x:')
print(x)
print('comprobación:')
print(comprobacion)

resultado x:
[ 1.   0.5 -2.   4.4 -0.5  3.3]
comprobación:
[15.59 44.02 18.05 43.96 28.24 34.49]


In [ ]:
x = resolverSistemaLU(A, b)
comprobacion = np.matmul(A, x)

print('resultado x:')
print(x)
print('comprobación:')
print(comprobacion)

resultado x:
[ 1.   0.5 -2.   4.4 -0.5  3.3]
comprobación:
[15.59 44.02 18.05 43.96 28.24 34.49]


In [ ]:
x = resolverSistemaPALU(A, b)
comprobacion = np.matmul(A, x)
print('resultado x:')
print(x)
print('comprobación:')
print(comprobacion)

resultado x:
[ 1.   0.5 -2.   4.4 -0.5  3.3]
comprobación:
[15.59 44.02 18.05 43.96 28.24 34.49]


In [ ]:
# print(A)
x = resolverJacobi(A, b, 100)
comprobacion = np.matmul(A, x)

print('resultado x:')
print(x)
print('comprobación:')
print(comprobacion)

[[ 0.         -0.43396226 -0.86792453 -0.50943396 -0.30188679 -0.41509434]
 [-0.30769231  0.         -0.73076923 -1.07692308 -0.43589744 -0.53846154]
 [-1.41666667 -2.33333333  0.         -0.70833333 -3.08333333 -1.625     ]
 [-1.36065574 -1.2295082  -1.50819672  0.         -0.85245902 -1.29508197]
 [-0.87755102 -1.20408163 -1.46938776 -0.53061224  0.         -1.63265306]
 [-1.83673469 -0.55102041 -1.         -0.97959184 -1.36734694  0.        ]]
Radio espectral:  5.350300193430739 converge: False
La matriz no converge a una solución


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [ ]:
x = resolverGaussSeidel(A, b, 50)
comprobacion = np.matmul(A, x)

print('resultado x:')
print(x)
print('comprobación:')
print(comprobacion)

Radio espectral:  2.7287055469547736 converge: False
La matriz no converge a una solución


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [ ]:
x = resolverSOR(A, b, 55, 1.3)
comprobacion = np.matmul(A, x)

print('resultado x:')
print(x)
print('comprobación:')
print(comprobacion)

Radio espectral:  3.5872275353930867 converge: False
La matriz no converge a una solución


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)